# Basic Page-Number Pagination

Validation notebook for page-number based pagination

In [ ]:
import os
import time
import json
import requests
from typing import Dict

# Modern data stack
import psycopg
import polars as pl

# Configuration - Auto-detect environment
ENVIRONMENT = os.getenv("NOETL_ENV", "localhost").lower()

if ENVIRONMENT == "kubernetes":
    DB_CONFIG = {
        "host": "postgres.postgres.svc.cluster.local",
        "port": "5432",
        "user": os.getenv("POSTGRES_USER", "demo"),
        "password": os.getenv("POSTGRES_PASSWORD", "demo"),
        "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
    }
    NOETL_SERVER_URL = "http://noetl.noetl.svc.cluster.local:8082"
else:
    DB_CONFIG = {
        "host": "localhost",
        "port": "54321",
        "user": os.getenv("POSTGRES_USER", "demo"),
        "password": os.getenv("POSTGRES_PASSWORD", "demo"),
        "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
    }
    NOETL_SERVER_URL = "http://localhost:8082"

TEST_PATH = "tests/pagination/basic"
POLL_INTERVAL = 2
MAX_WAIT = 60

print("✓ Configuration loaded")
print(f"  Environment: {ENVIRONMENT}")
print(f"  Server: {NOETL_SERVER_URL}")
print(f"  Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}")
print(f"  Test: {TEST_PATH}")

In [ ]:
def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={DB_CONFIG['host']} port={DB_CONFIG['port']} " \
                  f"dbname={DB_CONFIG['dbname']} user={DB_CONFIG['user']} " \
                  f"password={DB_CONFIG['password']}"
    return psycopg.connect(conn_string)

def query_to_polars(query: str) -> pl.DataFrame:
    """Execute query and return as Polars DataFrame"""
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            data = cur.fetchall()
    if not data:
        return pl.DataFrame(schema=columns)
    return pl.DataFrame({col: [row[i] for row in data] for i, col in enumerate(columns)})

print("✓ Database utilities loaded")

In [ ]:
def start_test() -> Dict:
    """Start pagination test"""
    url = f"{NOETL_SERVER_URL}/api/run/playbook"
    payload = {"path": TEST_PATH}
    
    print(f"Starting test: {TEST_PATH}")
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
    
    result = response.json()
    execution_id = result['execution_id']
    
    print(f"✓ Test started")
    print(f"  Execution ID: {execution_id}")
    print(f"  Status: {result['status']}")
    
    return result

test_result = start_test()
EXECUTION_ID = test_result['execution_id']

In [ ]:
def monitor_execution(execution_id: int):
    """Monitor test execution"""
    start_time = time.time()
    last_count = 0
    
    print(f"Monitoring execution {execution_id}...")
    print(f"{'Time':<6} {'Steps':<6} {'Status':<12} {'Events'}")
    print("-" * 50)
    
    while (time.time() - start_time) < MAX_WAIT:
        query = f"""
            SELECT event_type, COUNT(*) as count
            FROM noetl.event
            WHERE execution_id = {execution_id}
            GROUP BY event_type
        """
        df = query_to_polars(query)
        
        step_count = df.filter(pl.col('event_type') == 'step_completed')['count'].sum() or 0
        is_complete = df.filter(pl.col('event_type') == 'playbook_completed').height > 0
        is_failed = df.filter(pl.col('event_type') == 'playbook_failed').height > 0
        
        if step_count != last_count or is_complete or is_failed:
            elapsed = int(time.time() - start_time)
            status = "COMPLETED" if is_complete else ("FAILED" if is_failed else "RUNNING")
            total = df['count'].sum()
            print(f"{elapsed:<6} {step_count:<6} {status:<12} {total}")
            last_count = step_count
        
        if is_complete:
            print(f"\n✓ Test completed in {elapsed}s")
            return True
        elif is_failed:
            print(f"\n✗ Test failed after {elapsed}s")
            return False
        
        time.sleep(POLL_INTERVAL)
    
    print(f"\n⚠ Timeout after {MAX_WAIT}s")
    return False

success = monitor_execution(EXECUTION_ID)

In [ ]:
def show_execution_events(execution_id: int):
    """Display ordered table of events for an execution"""
    query = f"""
        SELECT 
            ROW_NUMBER() OVER (ORDER BY event_id) as seq,
            event_type,
            node_name,
            node_type,
            status,
            created_at
        FROM noetl.event
        WHERE execution_id = {execution_id}
        ORDER BY event_id
    """
    
    df = query_to_polars(query)
    
    print(f"\n📋 Events for Execution {execution_id}")
    print("=" * 120)
    print(f"{'#':<4} {'Event Type':<25} {'Node':<30} {'Type':<12} {'Status':<12} {'Created'}")
    print("-" * 120)
    
    for row in df.iter_rows(named=True):
        node_name = row['node_name'] or '-'
        node_type = row['node_type'] or '-'
        status = row['status'] or '-'
        created = str(row['created_at']) if row['created_at'] else '-'
        
        print(f"{row['seq']:<4} {row['event_type']:<25} {node_name:<30} "
              f"{node_type:<12} {status:<12} {created}")
    
    print("-" * 120)
    print(f"Total events: {len(df)}\n")

show_execution_events(EXECUTION_ID)

In [ ]:
print(f"\n🔍 Validation for Basic Page-Number Pagination")
print("=" * 80)

# Get final result from step
query = f"""
    SELECT result
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
    AND event_type = 'step_result'
    ORDER BY created_at DESC
    LIMIT 1
"""

with get_postgres_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        row = cur.fetchone()
        if row:
            result = row[0]
            print(f"✓ Final result retrieved")
            print(f"  Status: {result.get('status')}")
            print(f"  Total items: {result.get('total_items')}")
            
            # Validate
            expected_items = 35
            actual_items = result.get('total_items', 0)
            
            if actual_items == expected_items:
                print(f"\n🎉 SUCCESS! Fetched {actual_items} items (expected {expected_items})")
            else:
                print(f"\n❌ FAILED! Expected {expected_items} items, got {actual_items}")
        else:
            print("❌ No result found")

print("=" * 80)